In [31]:
import torch
import numpy as np
import random
import math

from torch.optim import SGD
from torch.optim.lr_scheduler import LinearLR
from torch.utils.data import DataLoader, TensorDataset

from models import MLP
from preprocess import preprocess_UCI_dataset

seed = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print ("Device: ", device)
print ("Seed: ", seed)

# set random seed
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

name_to_id_dict = {
    "wine_quality": 186,
    "abalone": 1,
    "liver_disorders": 60,
    "concrete_compressive_strength": 165
}

encoding_type = "one_hot"
test_size = 0.2
normalize_target = True

Device:  cuda
Seed:  42


In [32]:
dataset_name = "concrete_compressive_strength"

X_train, X_test, y_train, y_test, preprocessor, y_scaler = preprocess_UCI_dataset(
    name_to_id_dict[dataset_name], 
    encoding_type, 
    normalize_target,
    test_size, 
    seed
)
    
# convert to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)

X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

# summary of the dataset
print ("X_train shape: ", X_train.shape)
print ("y_train shape: ", y_train.shape)
print ("X_test shape: ", X_test.shape)
print ("y_test shape: ", y_test.shape)

# set batch size to 0.05 of the training dataset, then pick the closest from [4, 8, 16, 32, 64, 128]
batch_size = math.ceil(0.05 * X_train.shape[0])
valid_batch_sizes = [4, 8, 16, 32, 64, 128]
batch_size = min(valid_batch_sizes, key=lambda x: abs(x - batch_size))
print("Batch size:", batch_size)

train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
print ("Number of iterations per epoch:", len(train_dataloader))

Fetched dataset name: Concrete Compressive Strength
Numeric features:  ['Cement', 'Blast Furnace Slag', 'Fly Ash', 'Water', 'Superplasticizer', 'Coarse Aggregate', 'Fine Aggregate', 'Age']
Categorical features:  []
X_train shape:  torch.Size([824, 8])
y_train shape:  torch.Size([824, 1])
X_test shape:  torch.Size([206, 8])
y_test shape:  torch.Size([206, 1])
Batch size: 32
Number of iterations per epoch: 26


In [37]:
lr = 1e-3 #[1e-2. 1e-3, 1e-4, 1e-5]
epochs = 500 #[20, 50, 100, 500, 1000]
model = MLP(X_train.shape[-1], [32, 32], 1).to(device)
optimizer = SGD(model.parameters(), lr=lr, momentum=0.9)
lr_scheduler = LinearLR(optimizer, end_factor=0.1, start_factor=1, total_iters=epochs)

In [38]:
train_losses = []
test_losses = []

for epoch in range(epochs):

    model.train()
    train_loss = 0

    for i, (X_batch, y_batch) in enumerate(train_dataloader):
        y_pred = model(X_batch)
        loss = torch.nn.functional.mse_loss(y_pred, y_batch, reduction='mean')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    lr_scheduler.step()
    
    train_losses.append(train_loss/len(train_dataloader))

    model.eval()
    y_pred = model(X_test)
    test_loss = torch.mean((y_pred - y_test)**2).item()
    test_losses.append(test_loss)

    if (epoch+1) % ((epochs)//20) == 0:
        print (f"Epoch: {epoch+1} Train Loss: {train_loss/len(train_dataloader)} Test Loss: {test_loss}")

Epoch: 25 Train Loss: 0.26802275215203947 Test Loss: 0.24800094962120056
Epoch: 50 Train Loss: 0.16278743400023535 Test Loss: 0.17417755722999573
Epoch: 75 Train Loss: 0.13436446711421013 Test Loss: 0.1491585522890091
Epoch: 100 Train Loss: 0.11564916945420779 Test Loss: 0.13482382893562317
Epoch: 125 Train Loss: 0.1050636344231092 Test Loss: 0.126578226685524
Epoch: 150 Train Loss: 0.09871204111438531 Test Loss: 0.1239461824297905
Epoch: 175 Train Loss: 0.09446399601606223 Test Loss: 0.11753694713115692
Epoch: 200 Train Loss: 0.09069741889834404 Test Loss: 0.11521105468273163
Epoch: 225 Train Loss: 0.08715036124564134 Test Loss: 0.11354348063468933
Epoch: 250 Train Loss: 0.08424913868881188 Test Loss: 0.11344689130783081
Epoch: 275 Train Loss: 0.08173844046317615 Test Loss: 0.11142310500144958
Epoch: 300 Train Loss: 0.07942246502408615 Test Loss: 0.11094718426465988
Epoch: 325 Train Loss: 0.07802742476073596 Test Loss: 0.10960306972265244
Epoch: 350 Train Loss: 0.07635684254077765 Tes